<h1 style="text-align:center; font-size:200%;">NLP LAB 4</h1>
<h2 style="text-align:center; font-size:100%;">Réaliser par: Marouan Daghmoumi</h2>




<h1 style="text-align:center; font-size:150%;">2.Transformer</h1>




In [1]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
device

'cuda'

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [5]:
import os
import csv
from torch.utils.data import Dataset, DataLoader
class NewsDataset():
    def __init__(self, news_dataset_path='/kaggle/input/news-summary/'):
        super().__init__()

        news_summary_path = os.path.join(news_dataset_path, 'news_summary.csv')

        self.food_list = []
        self.end_of_text_token = ""

        with open(news_summary_path, encoding='utf-8', errors='replace') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')

            # Skip header
            next(csv_reader)

            for row in csv_reader:
                if len(row) < 5:  # Check if row has at least 5 elements
                    continue  # Skip this row if it doesn't

                headline = row[1]  # Assuming headlines are in the second column
                text = row[3]  # Assuming text is in the fourth column
                
                joke_str = f"headline: {headline} text: {text}{self.end_of_text_token}"
                self.food_list.append(joke_str)

    def __len__(self):
        return len(self.food_list)

    def __getitem__(self, item):
        return self.food_list[item]


In [7]:
dataset = NewsDataset()
news_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [8]:
!pip install transformers

In [10]:
BATCH_SIZE = 4
EPOCHS = 7
LEARNING_RATE = 3e-5
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 800

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
device

'cuda'

In [12]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=-1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_news_tens = None
models_folder = "trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx, news in enumerate(news_loader):
        
        #################### "Fit as many news sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        news_tens = torch.tensor(tokenizer.encode(news[0])).unsqueeze(0).to(device)
        # Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if news_tens.size()[1] > MAX_SEQ_LEN:
            continue
        
        # The first news sequence in the sequence
        if tmp_news_tens is None:
            tmp_news_tens = news_tens
            continue
        else:
            # The next news does not fit in so we process the sequence and leave the last news 
            # as the start for next sequence 
            if tmp_news_tens.size()[1] + news_tens.size()[1] > MAX_SEQ_LEN:
                work_news_tens = tmp_news_tens
                tmp_news_tens = news_tens
            else:
                # Add the news to sequence, continue and try to add more
                tmp_news_tens = torch.cat([tmp_news_tens, news_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it through the model ##################
            
        outputs = model(work_news_tens, labels=work_news_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step() 
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 100:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0
    
    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_medium_food_{epoch}.pt"))


EPOCH 0 started==============================
sum loss 932.9513549804688
EPOCH 1 started==============================
sum loss 880.8663940429688
EPOCH 2 started==============================
sum loss 814.1277465820312
EPOCH 3 started==============================
sum loss 773.2225952148438
EPOCH 4 started==============================
sum loss 755.0206909179688
EPOCH 5 started==============================
sum loss 734.057861328125
EPOCH 6 started==============================
sum loss 723.2421264648438


In [14]:
import os

# List files in the trained_models directory
models_folder = "/kaggle/working/trained_models"  # Assuming the trained_models folder is in the working directory
model_files = os.listdir(models_folder)

# Print the list of files
print(model_files)

['gpt2_medium_food_3.pt', 'gpt2_medium_food_2.pt', 'gpt2_medium_food_4.pt', 'gpt2_medium_food_0.pt', 'gpt2_medium_food_5.pt', 'gpt2_medium_food_6.pt', 'gpt2_medium_food_1.pt']


In [ ]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

MODEL_EPOCH = 4
models_folder = "trained_models"

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')

# Load model state dict
model_path = os.path.join(models_folder, f"gpt2_medium_food_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

# Output file path
news_output_file_path = f'generated_news_{MODEL_EPOCH}.txt'

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Set model to evaluation mode
model.eval()

# Remove existing output file if exists
if os.path.exists(news_output_file_path):
    os.remove(news_output_file_path)

with torch.no_grad():
    for i in range(1000):
        input_text = "NEWS:"
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate news
        output = model.generate(
            input_ids,
            max_length=800,
            num_return_sequences=1,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            repetition_penalty=1.0,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode and write generated news to file
        generated_news = tokenizer.decode(output[0], skip_special_tokens=True)
        with open(news_output_file_path, 'a') as f:
            f.write(f"{generated_news}\n\n")
